In [52]:
!pip install torch==1.3.1

In [0]:
from zipfile import ZipFile
import pathlib
from urllib.request import urlretrieve
import os
from os import remove
import os.path
import numpy as np
#import Augmentor as aug
import pandas as pd
import shutil as sh
import torch
import glob
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
FID_DIRECTORY = pathlib.Path("FID-300")
FID_LABELS = FID_DIRECTORY / "label_table.csv"
FID_SOURCE_URL = "https://fid.dmi.unibas.ch/FID-300.zip"
TRAIN_DIR = "FID-300/tracks_cropped/"
NUM_EPOCHS = 10

In [0]:
def process_images(valid_size=0.1):
    transformations = transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
    ])

    train_data = datasets.ImageFolder(TRAIN_DIR, transform=transformations)
    test_data = datasets.ImageFolder(TRAIN_DIR, transform=transformations)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,sampler=train_sampler, 
                                              batch_size=128)
    testloader = torch.utils.data.DataLoader(test_data,
                                             sampler=test_sampler, batch_size=1)
    return trainloader, testloader


In [0]:
def organize_files(label_df):
    """ Moves all pngs in tracked_cropped into subfolders by label (for PyTorch Image Folder """
    train_dir = pathlib.Path(TRAIN_DIR)

    files = glob.glob(str(train_dir / "*.jpg"))
    for i in range(len(files)):
        f = pathlib.Path(files[i])
        fname = f.name
        id = int(f.stem)
        label = label_df["label"].iloc[id-1]
        new_dir = train_dir / str(label)
        new_dir.mkdir(exist_ok=True)
        new_file = new_dir / fname

        sh.move(f, new_file)

In [0]:
def load_labels():
    labels = pd.read_csv(FID_LABELS, delimiter=",", header=None,
                         dtype=np.dtype(int), names=['id', 'label'])
    # print(len(pd.unique(labels['label'])))
    return labels


def fetch_FID_300_data():
    """Downloads and extracts FID-300 data to a local folder"""
    if FID_DIRECTORY.exists():
        print("FID-300 Database already exists")
        return
    print("Downloading FID_300")
    local_file, _ = urlretrieve(FID_SOURCE_URL)
    with ZipFile(local_file) as archive:
        print("Extracting FID-300")
        archive.extractall()
    remove(local_file)


In [0]:
def preprocess_data():
    fetch_FID_300_data()
    labels = load_labels()
    organize_files(labels)
    train_load, test_load = process_images()
    return train_load, test_load

In [0]:
class SimpleCNN(torch.nn.Module):
    #Our batch shape for input x is (3, 32, 32)
    def __init__(self):
        super(SimpleCNN, self).__init__()
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        #64 input features, 130 output features for our 130 defined classes
        self.fc2 = torch.nn.Linear(64, 130)
        
    def forward(self, x):
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        #Size changes from (18, 16, 16) to (1, 4608)
        x = x.view(-1, 18 * 16 *16)
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [0]:
class AverageBase(object):
    def __init__(self, value=0):
        self.value = float(value) if value is not None else None
    def __str__(self):
        return str(round(self.value, 4))
    def __repr__(self):
        return self.value
    def __format__(self, fmt):
        return self.value.__format__(fmt)
    def __float__(self):
        return self.value

class RunningAverage(AverageBase):
    """ Keeps track of a cumulative moving average (CMA). """
    def __init__(self, value=0, count=0):
        super(RunningAverage, self).__init__(value)
        self.count = count
    def update(self, value):
        self.value = (self.value * self.count + float(value))
        self.count += 1
        self.value /= self.count
        return self.value

class MovingAverage(AverageBase):
    """ An exponentially decaying moving average (EMA). """
    def __init__(self, alpha=0.99):
        super(MovingAverage, self).__init__(None)
        self.alpha = alpha
    def update(self, value):
        if self.value is None:
            self.value = float(value)
        else:
            self.value = self.alpha * self.value + (1 - self.alpha) * float(value)
        return self.value

In [0]:
from IPython.display import HTML, display

class ProgressMonitor(object):
    """ Custom IPython progress bar for training """
    tmpl = """
        <p>Loss: {loss:0.4f}   {value} / {length}</p>
        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
    """
    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)
    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))
    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

In [0]:
def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)

def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [0]:
def train(train_loader, test_loader, model, num_epochs=10, first_epoch=1):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    valid_losses = []

    for epoch in range(first_epoch, first_epoch + num_epochs):
        print('Epoch', epoch)
        # train phase
        model.train()
        # create a progress bar
        progress = ProgressMonitor(length=len(train_loader))
        train_loss = MovingAverage()
        for batch, targets in train_loader:
            # Move the training data to the GPU
            batch = batch.to(device)
            targets = targets.to(device)

            # clear previous gradient computation
            optimizer.zero_grad()
            # forward propagation
            predictions = model(batch)
            # calculate the loss
            loss = criterion(predictions, targets)
            # backpropagate to compute gradients
            loss.backward()
            # update model weights
            optimizer.step()
            # update average loss
            train_loss.update(loss)
            # update progress bar
            progress.update(batch.shape[0], train_loss)

        print('Training loss:', train_loss)
        train_losses.append(train_loss.value)

        # validation phase
        model.eval()
        valid_loss = RunningAverage()
        # keep track of predictions
        y_pred = []
        target = test_loader;
        with torch.no_grad():
            for batch, targets in test_loader:
                # Move the training batch to the GPU
                batch = batch.to(device)
                targets = targets.to(device)
                # forward propagation
                predictions = model(batch)
                # calculate the loss
                loss = criterion(predictions, targets)
                # update running loss value
                valid_loss.update(loss)
                # save predictions
                y_pred.extend(predictions.argmax(dim=1).cpu().numpy())
                target = targets.cpu()
        print('Validation loss:', valid_loss)
        valid_losses.append(valid_loss.value)
        # Calculate validation accuracy
        y_pred = torch.tensor(y_pred, dtype=torch.int64)
        accuracy = torch.mean((y_pred == target).float())
        print('Validation accuracy: {:.4f}%'.format(float(accuracy) * 100))
        print()
        # Save a checkpoint
        #checkpoint_filename = 'checkpoints/mnist-{:03d}.pkl'.format(epoch)
        #save_checkpoint(optimizer, model, epoch, checkpoint_filename)
    return train_losses, valid_losses, y_pred

In [0]:
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleCNN() 
    model.to(device)
    return device, model

In [65]:
train_load, test_load = preprocess_data()
device, model = load_model()
train_losses, valid_losses, y_pred = train(train_load, test_load, model, num_epochs=1000)

FID-300 Database already exists
Epoch 1


Training loss: 4.8915
Validation loss: 4.8756
Validation accuracy: 0.0000%

Epoch 2


Training loss: 4.8711
Validation loss: 4.8249
Validation accuracy: 0.0000%

Epoch 3


Training loss: 4.8699
Validation loss: 4.8228
Validation accuracy: 0.0000%

Epoch 4


Training loss: 4.8867
Validation loss: 4.8119
Validation accuracy: 0.0000%

Epoch 5


...

Training loss: 1.7261
Validation loss: 8.818
Validation accuracy: 0.0000%

Epoch 997


Training loss: 1.8476
Validation loss: 9.8318
Validation accuracy: 3.3333%

Epoch 998


Training loss: 1.8223
Validation loss: 10.2084
Validation accuracy: 3.3333%

Epoch 999


Training loss: 1.5629
Validation loss: 9.3994
Validation accuracy: 0.0000%

Epoch 1000


Training loss: 1.5195
Validation loss: 10.6151
Validation accuracy: 0.0000%

